# Recursive Descent Parser

A **recursive descent parser** is a kind of top-down parser built from a set of mutually recursive procedures where each such procedure implements one of the nonterminals of the grammar.

Thus the structure of the resulting program closely mirrors that of the grammar it recognizes. A predictive parser is a recursive descent parser that does not require backtracking. Predictive parsing is possible only for the class of *LL grammars*, which are the context-free grammars for which there exists some positive integer *k* that allows a recursive descent parser to decide which production to use by examining only the next *k* tokens of input.


In [1]:
// Load some auxiliary tools
#load "grammartools.fsx"
open CSCI374.GrammarTools
open CSCI374.ParserTypes

Let's defined a class for a lexical analyzer `Tokenizer` which will facilitate operations with the token stream.
- All necessary types are defined in [grammartools.fsx](grammartools.fsx) script (e.g `PRODUCTION`, `TOKEN`, and etc.)

In [2]:
type Tokenizer(grammar: PRODUCTION [], verbose: bool) =
    let mutable inputState = []
    let mutable curentToken = INVALID

    // Access to the
    member this.CurrentToken = curentToken    
    member this.NextToken() =
        let tkn, input = CSCI374.Lexer.token inputState
        inputState <- input
        curentToken <- tkn
        this
    
    member this.InputState
        with set(str) = inputState <- Seq.toList str
    member this.IsVerbose = verbose
    member this.PrintRule ruleIdx =
        printGrammarRule false grammar ruleIdx // print rule
    new(grammar) = Tokenizer(grammar, false)
    

/// This infix operator function provides verbose output while calling
/// a particular production rule
let (==>) (cnxt:Tokenizer) (prod:Tokenizer->Tokenizer) =
    if cnxt.IsVerbose then
        printfn "Enter <%A> with token `%A`" prod cnxt.CurrentToken
    let nextcnxt = prod cnxt
    if cnxt.IsVerbose then
        printfn "Exit <%A> with token `%A`" prod cnxt.CurrentToken
    nextcnxt
    
/// This infix operator function will allow to print a production rule
/// call `cnxt @ 2` will print second grammar rule
let (@) (cnxt:Tokenizer) ruleIdx =
    cnxt.PrintRule ruleIdx
    cnxt

Let's write a recursive descent parser for a following grammar
```
S → T | ( S + T )
T → a
```

This is LL grammar as it doesn't have left recursion and every production passes pairwise disjointness test.

The [grammartools.fsx](grammartools.fsx) script provides a function `parseGrammarString` that parses a string representation of the grammar in to array of production rules that we can for our parser.

In [3]:
let grammar = parseGrammarString """
    S → T | ( S + T )
    T → a
"""
printfn "%A" grammar

[|(NonTerminal S, [NonTerminal T]);
  (NonTerminal S,
   [Terminal LPAR; NonTerminal S; Terminal PLUS; NonTerminal T; Terminal RPAR]);
  (NonTerminal T, [Terminal A])|]


In [4]:
// Show grammar rules
printGrammar grammar

Grammar:
1: S → T
2: S → (S+T)
3: T → a



How do we write a recursive descent parser? There is a recursive function for each nonterminal in the grammar, which can parse sentences that can be generated by that nonterminal:

1. Assume that the grammar rule has only one right hand side (RHS) production (e.g. `T → a`):
    - For each terminal symbol in the RHS, compare it with the next input token, and if they match, continue, else there is an error. We call this function `Match`.
    - For each nonterminal symbol in the RHS, call its associated parsing function. We call these functions `Prod`.
2. Assume that the grammar rule has more than one right hand side (RHS) requires an initial process to determine which RHS it is to parse (e.g. `S → T | ( S + T )`):
    - The correct RHS is chosen on the basis of the next token of input (the lookahead)
    - The next token is compared with the first token that can be generated by each RHS until a match is found
    - If no match is found, it is a syntax error

Let's write these recursive function for the above grammar.
- F# provides a specific syntax for [mutually recursive functions](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/functions/recursive-functions-the-rec-keyword#mutually-recursive-functions)

In [5]:
/// Start with a production S → T | ( S + T )
/// Let's enumerate rules as follows
/// 1: S → T
/// 2: S → ( S + T )
/// because RHS productions are pairwise disjoint, the rule #2 is always starts with matching `(`
let rec ProdS (cnxt:Tokenizer) =
    // check the current token is `(` then select rule #2
    if cnxt.CurrentToken = LPAR then
        // 2: S -> ( S + T )
        cnxt.NextToken() @(2)==> ProdS ==> Match PLUS ==> ProdT ==> Match RPAR 
    else
        // 1: S -> T
        cnxt @(1)==> ProdT 

/// The function for production T → a is straight forward: match nonterminal `a`
and ProdT (cnxt:Tokenizer) =
    // 3: T -> a
    cnxt @(3)==> Match A

/// For each terminal symbol compare it with a current token
/// and if they match, continue with the next token, else there is an error
and Match term cnxt =
    if cnxt.IsVerbose then printfn "Match %A with %A" term cnxt.CurrentToken
    // if we matched the current token with a terminal symbol
    if term = cnxt.CurrentToken then
        cnxt.NextToken() // read next token
    else
        failwith (sprintf "Cannot match symbol `%A` with `%A`" term cnxt.CurrentToken)
    
/// Start parsing by calling starting symbol function
let parser (cnxt:Tokenizer) :Tokenizer =    
    // Read token and pass it to the function for S rule
    cnxt.NextToken() ==> ProdS

The string to parse is `((a+a)+a)`. We run this string through the lexical analyzer to see a tokenized output.

In [6]:
let inputString = "((a+a)+a)"
inputString |> Seq.toList |> CSCI374.Lexer.tokenize |> printfn "%A"

[LPAR; LPAR; A; PLUS; A; RPAR; PLUS; A; RPAR; END]


Create a tokenizer object and begin parsing. We should be able to see a sequence of grammar rules required to parse the above expression.

In [7]:
Tokenizer(grammar, InputState=inputString) |> parser |> ignore

2: S → (S+T)
2: S → (S+T)
1: S → T
3: T → a
3: T → a
3: T → a


Let's check if a string is not generated by our grammar: `((a+)+a)`. In addition, we turn on more verbose parser output by setting second parameter of `Tokenizer` to `true.

In [8]:
Tokenizer(grammar, true, InputState="((a+)+a)") |> parser |> ignore

Enter <<fun:it@1-1>> with token `LPAR`
2: S → (S+T)
Enter <<fun:ProdS@10>> with token `LPAR`
2: S → (S+T)
Enter <<fun:ProdS@10>> with token `A`
1: S → T
Enter <<fun:ProdS@13-4>> with token `A`
3: T → a
Enter <<fun:ProdT@18>> with token `A`
Match A with A
Exit <<fun:ProdT@18>> with token `PLUS`
Exit <<fun:ProdS@13-4>> with token `PLUS`
Exit <<fun:ProdS@10>> with token `PLUS`
Enter <<fun:ProdS@10-1>> with token `PLUS`
Match PLUS with PLUS
Exit <<fun:ProdS@10-1>> with token `RPAR`
Enter <<fun:ProdS@10-2>> with token `RPAR`
3: T → a
Enter <<fun:ProdT@18>> with token `RPAR`
Match A with RPAR


Unhandled Exception: Cannot match symbol `A` with `RPAR`